# Chemical Intervention on Rat

Experiment Narrative:
---------------------

Male Fisher F344 rats purchased from Charles River were treated with 3 commonly used painkillers, namely acetylsalicilic acid, acetaminophen and ibuprofen, at 2 distinct dose levels, delivered per os.
Equal number of animals (n=5) were allocated to each group defined by a compound, dose level and duration post exposure combination.
Following sacrifice performed by cervical dislocation and exsanguination preceded by anesthesia (ketamine and xylazine solution), blood and kidney specimens were collected .
Blood samples were collected at sacrifice time and immediately place in RNALater. Blood Metabolites were separated in water soluble and lipophilic fraction. PBMC were collected, RNA extracted and transcription profiling performed using paired-end RNA-Seq on Illumina sequencing platform using a Illumina HiSeq 2000 instrument. Total RNA were also extracted from Kidney samples and mRNA sequencing using the same approach.
Metabolite profiling was performed on the polar fraction only, using flow injection analysis mass spectrometry on an Agilent 6550 iFunnel Q-TOF Mass Sprectrometry platform. Each fraction was injected twice and data was acquired in both ionization modes (positive mode and negative mode).
Raw data file were saved in native instrument format and later converted to HUPO PSI standard format for mass spectrometry.


In [1]:
from ipywidgets import (RadioButtons, VBox, HBox, Layout, Label, Checkbox, Text, IntSlider)
from qgrid import show_grid
label_layout = Layout(width='100%')
from isatools.create.models import *
from isatools.model import Investigation
from isatools.isatab import dump_tables_to_dataframes as dumpdf
import qgrid
qgrid.nbinstall(overwrite=True)

## Study Design Description and Treatment Plans

### Study design type

Based on the textual description provided, specify the type of study being conducted, identify the different independent variables and their respective levels.

### Intervention study

If specifying an intervention study, please answer the following:
 - Are study subjects exposed to a single intervention or to multiple intervention?
  
 
*Note: if you chose 'observation' as the study design type, the following choices will be disabled and you should skip to the Observation study section*
 

In [2]:
study_design = InterventionStudyDesign()

In [3]:
design_type = '' 
print('Design type: {}'.format(design_type))

Design type: factorial


#### Factorial design - intervention types

In the context of the experiment, identify the independent variables and their associated levels.
How many treatment group are defined? 


In [4]:
level_uis = []
agent_levels = Text(
        description='Agent:',
        value='',
        placeholder='',

    )
dose_levels = Text(
        description='Dose levels:',
        value='',
        placeholder='',

    )
duration_of_exposure_levels = Text(
        description='Duration post exposure:',
        value='',
        placeholder='',
)

# Computing the Number of Unique Treatment Groups/Study Groups: 

Using the ISA-API create mode, determine the total number of treatment groups.

In [5]:
factory = TreatmentFactory(intervention_type=INTERVENTIONS['CHEMICAL'], factors=BASE_FACTORS)
for agent_level in agent_levels.value.split(','):
    factory.add_factor_value(BASE_FACTORS[0], agent_level.strip())
for dose_level in dose_levels.value.split(','):
    factory.add_factor_value(BASE_FACTORS[1], dose_level.strip())
for duration_of_exposure_level in duration_of_exposure_levels.value.split(','):
    factory.add_factor_value(BASE_FACTORS[2], duration_of_exposure_level.strip())
    
print('Number of study groups (treatment groups): {}'.format(len(factory.compute_full_factorial_design())))

treatment_sequence = TreatmentSequence(ranked_treatments=factory.compute_full_factorial_design())

Number of study groups (treatment groups): 18


# Setting Study Group Sizes:

Next, specify if all study groups have the same size (in other words, do they have the same number of subjects?)

What type of replication this is? 

From the following list of terms, select which that apply to the particular study:

In [6]:
group_balanced = RadioButtons(options=['Balanced', 'Unbalanced'], value='Balanced', disabled=False)
VBox([Label('Are study groups balanced?', layout=label_layout), group_balanced])

In [7]:
group_size = IntSlider(value=1, min=0, max=100, step=1, description='Group size:', disabled=False, continuous_update=False, orientation='horizontal', readout=True, readout_format='d')
group_size

In [8]:
group_size.value="1"
plan = SampleAssayPlan(group_size=group_size.value)

# Establishing the sample collection plan

For each study subject, determine the number of unique specimens collected over the course of the study.
Provide the number of subject per study group:

In [9]:
collection_plan = {"biomaterial":1} #dictionary key should be a 'biological specimen' (e.g. brain), dictionary value should be an integer

for key in collection_plan:
    sample_type = key
    collected_sample_type_nb = collection_plan[key]
    #print(key, collected_sample_type_nb)
    plan.add_sample_type(sample_type)
    plan.add_sample_plan_record(sample_type,collected_sample_type_nb)
    isa_object_factory = IsaModelObjectFactory(plan, treatment_sequence)

How many times each of the samples have been collected?

## Create an ISA Study Table reflecting the study design and the sampling plan:

In the following section, the task is to build ISA objects from the sample plan and render the study-sample table.


*Check state of the Sample Assay Plan after entering sample planning information:*

In [10]:
isa_investigation = Investigation(identifier='inv_dtp-exercise-1')
isa_study = isa_object_factory.create_study_from_plan()
isa_study.filename = 's_study.txt'
isa_investigation.studies = [isa_study]
dataframes = dumpdf(isa_investigation)
sample_table = next(iter(dataframes.values()))
show_grid(sample_table)

In [11]:
print('Total rows generated: {}'.format(len(sample_table)))

Total rows generated: 180


# Saving key study design information as a JSON document.

This section is meant to show how to serialize key study design descriptor as a summarized version. 
Why is this relevant ?
How would you use this feature. List 3 possible use.

In [12]:
import json
from isatools.create.models import SampleAssayPlanEncoder
print(json.dumps(plan, cls=SampleAssayPlanEncoder, sort_keys=True, indent=4, separators=(',', ': ')))

{
    "assay_plan": [],
    "assay_types": [],
    "group_size": 5,
    "sample_plan": [
        {
            "sample_type": "Kidney",
            "sampling_size": 1
        },
        {
            "sample_type": "Blood",
            "sampling_size": 1
        }
    ],
    "sample_qc_plan": [],
    "sample_types": [
        "Blood",
        "Kidney"
    ]
}


## Documenting the Assay and Data Acquisition Plans:

From the textual description, identify the Response variables / Dependent Variables

### Select assay technology type to map to sample type from sample plan. Check against ISA configuration to obtain vetted values for Measurement Type and Technology type.
ISA configurations can be accessed from 

In [ ]:
assay_types = set()

assay_type1 = AssayType(measurement_type='', technology_type='')
assay_type2 = AssayType(measurement_type='', technology_type='')

assay_types.add(assay_type1)
assay_types.add(assay_type2)
for x in assay_types:
        print(x.measurement_type.term," using ", x.technology_type.term)


metabolite profiling  using  mass spectrometry
transcription profiling  using  nucleotide sequencing


### Assay Plan Modifiers (underlying workflow graph modifications)

In [14]:
ngs_technical_replicates = IntSlider(value=1, min=0, max=5, step=1, description='Technical repeats:', disabled=False, continuous_update=False, orientation='horizontal', readout=True, readout_format='d')
ngs_technical_replicates


In [15]:
ms_tech_rep = 1

## Generate ISA model objects from the assay plan and render the assay table

*Check state of Sample Assay Plan after entering assay plan information:*

In [16]:
sequencing_instruments = set()
sequencing_instruments.add('<some value here>') # check the experiment description to obtain value.

top_mods_seq = DNASeqAssayTopologyModifiers(technical_replicates=ngs_technical_replicates.value, instruments=sequencing_instruments)

print('Technical replicates: {}'.format(top_mods_seq.technical_replicates))

assay_type1.topology_modifiers = top_mods_seq

plan.add_assay_type(assay_type1)
plan.add_assay_plan_record("<some value here>", assay_type1)  # check the experiment description to obtain value.
assay_plan = next(iter(plan.assay_plan))

print('Added assay plan: {0} -> {1}/{2}'.format(assay_plan[0].value.term, assay_plan[1].measurement_type.term, assay_plan[1].technology_type.term))
if len(top_mods_seq.instruments) > 0:
    print('Instruments: {}'.format(list(top_mods_seq.instruments)))

Technical replicates: 1
Added assay plan: Kidney -> transcription profiling/nucleotide sequencing
Instruments: ['Illumina HiSeq 2000']


In [17]:
print(json.dumps(plan, cls=SampleAssayPlanEncoder, sort_keys=True, indent=4, separators=(',', ': ')))

{
    "assay_plan": [
        {
            "assay_type": {
                "measurement_type": "transcription profiling",
                "technology_type": "nucleotide sequencing",
                "topology_modifiers": {
                    "distinct_libraries": 0,
                    "instruments": [
                        "Illumina HiSeq 2000"
                    ],
                    "technical_replicates": 1
                }
            },
            "sample_type": "Kidney"
        }
    ],
    "assay_types": [
        {
            "measurement_type": "transcription profiling",
            "technology_type": "nucleotide sequencing",
            "topology_modifiers": {
                "distinct_libraries": 0,
                "instruments": [
                    "Illumina HiSeq 2000"
                ],
                "technical_replicates": 1
            }
        }
    ],
    "group_size": 5,
    "sample_plan": [
        {
            "sample_type": "Kidney",
            "sa

In [18]:

chromatography_instruments = set()
ms_instruments = set()
injection_modes = set()
acquisition_modes = set()

ms_instruments.add('<some value here>') # check the experiment description to obtain value.
injection_modes.add('<some value here>')  # check the experiment description to obtain value.
acquisition_modes.add('<some value here>') # check the experiment description to obtain value.
acquisition_modes.add('<some value here>') # check the experiment description to obtain value.
top_mods_ms = MSAssayTopologyModifiers(technical_replicates=ms_tech_rep, injection_modes=injection_modes, acquisition_modes=acquisition_modes, instruments=ms_instruments, chromatography_instruments=chromatography_instruments)
assay_type2.topology_modifiers = top_mods_ms

if len(top_mods_ms.chromatography_instruments) > 0:
    print('Chromatography instruments: {}'.format(list(top_mods_ms.chromatography_instruments)))
else:
    print('no chromatography used or no information supplied')

if len(top_mods_ms.instruments) > 0:
    print('Data acquisition instruments: {}'.format(list(top_mods_ms.instruments)))    
if len(top_mods_ms.injection_modes) > 0:
    print('Injection modes: {}'.format(list(top_mods_ms.injection_modes)))
if len(top_mods_ms.acquisition_modes) > 0:
    print('Acquisition modes: {}'.format(list(top_mods_ms.acquisition_modes)))


plan.add_assay_type(assay_type2)
plan.add_assay_plan_record("Blood", assay_type2)

assay_plan = next(iter(plan.assay_plan))



no chromatography used or no information supplied
Data acquisition instruments: ['Waters']
Injection modes: ['FIA']
Acquisition modes: ['positive', 'negative']


In [19]:
print(json.dumps(plan, cls=SampleAssayPlanEncoder, sort_keys=True, indent=4, separators=(',', ': ')))

{
    "assay_plan": [
        {
            "assay_type": {
                "measurement_type": "transcription profiling",
                "technology_type": "nucleotide sequencing",
                "topology_modifiers": {
                    "distinct_libraries": 0,
                    "instruments": [
                        "Illumina HiSeq 2000"
                    ],
                    "technical_replicates": 1
                }
            },
            "sample_type": "Kidney"
        },
        {
            "assay_type": {
                "measurement_type": "metabolite profiling",
                "technology_type": "mass spectrometry",
                "topology_modifiers": {
                    "acquisition_modes": [
                        "negative",
                        "positive"
                    ],
                    "chromatography_instruments": [],
                    "injection_modes": [
                        "FIA"
                    ],
                    "

In [20]:
isa_investigation.studies = [isa_object_factory.create_assays_from_plan()]
for assay in isa_investigation.studies[-1].assays:
    print('Assay generated: {0}, {1} samples, {2} processes, {3} data files'
          .format(assay.filename, len(assay.samples), len(assay.process_sequence), len(assay.data_files)))
dataframes = dumpdf(isa_investigation)

A protocol with name "metabolite extraction" has already been declared in the study
Assay generated: a_Kidney_dnaseq_Illumina HiSeq 2000_assay.txt, 90 samples, 270 processes, 90 data files
Assay generated: a_Blood_ms_FIA_positive_assay.txt, 90 samples, 270 processes, 180 data files
Assay generated: a_Blood_ms_FIA_negative_assay.txt, 90 samples, 270 processes, 180 data files


In [21]:
show_grid(dataframes[next(iter(dataframes.keys()))])

In [24]:
show_grid(dataframes['a_<specimen type>_ms_<injection mode>_<acquisition mode>_assay.txt'])

In [25]:
show_grid(dataframes['a__<specimen type>_ms_<injection mode>_<acquisition mode>_assay.txt'])